In [1]:
# Install dependencies
%pip install numpy
%pip install scikit-learn


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip3.11 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Normalize and assign the data to classes to load in batches for the model

In [5]:
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

# Quick normalization first
def quick_normalize(game_objects):
    print("Starting quick normalization...")
    scaler = MinMaxScaler()
    
    # Collect all data to fit the scaler
    print("Collecting all data to fit scaler...")
    all_data = []
    for game in game_objects:
        all_data.append(game.matchups)
        all_data.append(game.team_history)
        all_data.append(game.opponent_history)
    
    # Concatenate all data and fit the scaler
    print("Fitting scaler...")
    combined_data = pd.concat(all_data)
    scaler.fit(combined_data)
    
    # Now transform each game's data using the fitted scaler
    print("Transforming data...")
    for i, game in enumerate(game_objects):
        if i % 1000 == 0:
            print(f"Normalizing game {i}/{len(game_objects)}")
            
        # Normalize game_date
        baseline_date = pd.to_datetime("2000-01-01")
        game.game_date = (pd.to_datetime(game.game_date) - baseline_date).days
        
        # Transform each sequence using the same scaler
        game.matchups = pd.DataFrame(
            scaler.transform(game.matchups),
            columns=game.matchups.columns,
            index=game.matchups.index
        )
        game.team_history = pd.DataFrame(
            scaler.transform(game.team_history),
            columns=game.team_history.columns,
            index=game.team_history.index
        )
        game.opponent_history = pd.DataFrame(
            scaler.transform(game.opponent_history),
            columns=game.opponent_history.columns,
            index=game.opponent_history.index
        )
    
    return game_objects, scaler

# Classes for PyTorch handling
class GameData:
    def __init__(self, game_id, game_date, is_regular_season, is_playoffs, 
                 is_pre_season, matchups, team_history, opponent_history, target):
        self.game_id = game_id
        self.game_date = game_date
        self.is_regular_season = is_regular_season
        self.is_playoffs = is_playoffs
        self.is_pre_season = is_pre_season
        self.matchups = matchups
        self.team_history = team_history
        self.opponent_history = opponent_history
        self.target = target

    def to_tensor_dict(self):
        tensor_dict = {
            'matchups_tensor': torch.FloatTensor(self.matchups.values),
            'team_history_tensor': torch.FloatTensor(self.team_history.values),
            'opponent_history_tensor': torch.FloatTensor(self.opponent_history.values),
            'matchups_lengths': torch.LongTensor([len(self.matchups)]),
            'team_history_lengths': torch.LongTensor([len(self.team_history)]),
            'opponent_history_lengths': torch.LongTensor([len(self.opponent_history)]),
            'target': torch.FloatTensor([self.target]),
            'game_type': torch.FloatTensor([
                self.is_regular_season,
                self.is_playoffs,
                self.is_pre_season
            ])
        }
        return tensor_dict

    @staticmethod
    def collate_batch(batch):
        tensor_dicts = [game.to_tensor_dict() for game in batch]
        batch_dict = {}
        
        # No need for lengths or padding since sequences are fixed length
        for key in ['matchups_tensor', 'team_history_tensor', 'opponent_history_tensor']:
            batch_dict[key] = torch.stack([d[key] for d in tensor_dicts])
        
        batch_dict['game_type'] = torch.stack([d['game_type'] for d in tensor_dicts])
        batch_dict['target'] = torch.cat([d['target'] for d in tensor_dicts])
        
        return batch_dict

class NBAGamesDataset(Dataset):
    def __init__(self, game_objects):
        self.game_objects = game_objects
    
    def __len__(self):
        return len(self.game_objects)
    
    def __getitem__(self, idx):
        return self.game_objects[idx]

Normalize the data and save to pkl

In [8]:
with open('rnn_game_objects.pkl', 'rb') as f:
    game_objects = pickle.load(f)

# Quick normalize
normalized_games, scaler = quick_normalize(game_objects)

# Split into train/val/test
game_dates = np.array([game.game_date for game in normalized_games])
sorted_indices = np.argsort(game_dates)
normalized_games = [normalized_games[i] for i in sorted_indices]

n_train = int(0.8 * len(normalized_games))
n_val = int(0.10 * len(normalized_games))

train_games = normalized_games[:n_train]
val_games = normalized_games[n_train:n_train + n_val]
test_games = normalized_games[n_train + n_val:]

# Create datasets and dataloaders
batch_size = 32
train_dataset = NBAGamesDataset(train_games)
val_dataset = NBAGamesDataset(val_games)
test_dataset = NBAGamesDataset(test_games)

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=GameData.collate_batch
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=GameData.collate_batch
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=GameData.collate_batch
)

# Save everything
with open('game_stats_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

processed_data = {
    'train_dataset': train_loader.dataset.game_objects,
    'val_dataset': val_loader.dataset.game_objects,
    'test_dataset': test_loader.dataset.game_objects
}

with open('processed_game_objects.pkl', 'wb') as f:
    pickle.dump(processed_data, f)

# Print information
print(f"\nNumber of training batches: {len(train_loader)}")
print(f"Number of validation batches: {len(val_loader)}")
print(f"Number of test batches: {len(test_loader)}")

Starting quick normalization...
Fitting scaler...
Transforming data...
Normalizing game 0/28803
Normalizing game 1000/28803
Normalizing game 2000/28803
Normalizing game 3000/28803
Normalizing game 4000/28803
Normalizing game 5000/28803
Normalizing game 6000/28803
Normalizing game 7000/28803
Normalizing game 8000/28803
Normalizing game 9000/28803
Normalizing game 10000/28803
Normalizing game 11000/28803
Normalizing game 12000/28803
Normalizing game 13000/28803
Normalizing game 14000/28803
Normalizing game 15000/28803
Normalizing game 16000/28803
Normalizing game 17000/28803
Normalizing game 18000/28803
Normalizing game 19000/28803
Normalizing game 20000/28803
Normalizing game 21000/28803
Normalizing game 22000/28803
Normalizing game 23000/28803
Normalizing game 24000/28803
Normalizing game 25000/28803
Normalizing game 26000/28803
Normalizing game 27000/28803
Normalizing game 28000/28803

Number of training batches: 721
Number of validation batches: 90
Number of test batches: 91


Model architecture for RNN

In [9]:
import torch
import torch.nn as nn

class PredictorRNN(nn.Module):
    def __init__(self, feature_names, hidden_size=128, num_layers=2, dropout=0.2):
        super().__init__()
        self.feature_names = feature_names
        self.input_size = len(feature_names)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM networks for each sequence type
        self.matchups_lstm = nn.LSTM(
            input_size=self.input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=False
        )
        
        self.team_history_lstm = nn.LSTM(
            input_size=self.input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=False
        )
        
        self.opponent_history_lstm = nn.LSTM(
            input_size=self.input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=False
        )
        
        # Game type embedding
        self.game_type_proj = nn.Linear(3, hidden_size)
        
        # Combination layer
        combined_size = (hidden_size * 3) + hidden_size  # 3 LSTMs + game type
        
        self.classifier = nn.Sequential(
            nn.Linear(combined_size, hidden_size * 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size * 2, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, 1),
            nn.Sigmoid()
        )

    def forward(self, batch_dict):
        # Process matchups (always length 5)
        _, (matchups_hidden, _) = self.matchups_lstm(batch_dict['matchups_tensor'])
        matchups_hidden = matchups_hidden[-1]  # Take last layer's hidden state
        
        # Process team history (always length 10)
        _, (team_hidden, _) = self.team_history_lstm(batch_dict['team_history_tensor'])
        team_hidden = team_hidden[-1]
        
        # Process opponent history (always length 10)
        _, (opponent_hidden, _) = self.opponent_history_lstm(batch_dict['opponent_history_tensor'])
        opponent_hidden = opponent_hidden[-1]
        
        # Process game type
        game_type_embedded = self.game_type_proj(batch_dict['game_type'])
        
        # Combine all features
        combined = torch.cat([
            matchups_hidden,
            team_hidden,
            opponent_hidden,
            game_type_embedded
        ], dim=1)
        
        # Make prediction and ensure output is between 0 and 1
        output = self.classifier(combined)
        output = torch.clamp(output, 0, 1)  # Add this line
        
        return output
    
    def train_model(self, train_loader, val_loader, num_epochs=50, learning_rate=0.001, patience=5):
        criterion = nn.BCELoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5)
        
        best_val_loss = float('inf')
        best_model = None
        patience_counter = 0
        
        for epoch in range(num_epochs):
            # Training phase
            self.train()
            total_train_loss = 0
            correct_predictions = 0
            total_predictions = 0
            
            for batch in train_loader:
                optimizer.zero_grad()
                outputs = self(batch)
                # Reshape target to match output shape
                target = batch['target'].view(-1, 1)  # Change this line
                loss = criterion(outputs, target)
                
                loss.backward()
                optimizer.step()
                
                total_train_loss += loss.item()
                predictions = (outputs >= 0.5).float()
                correct_predictions += (predictions == target).sum().item()  # Update this line
                total_predictions += target.size(0)
            
            avg_train_loss = total_train_loss / len(train_loader)
            train_accuracy = correct_predictions / total_predictions
            
            # Validation phase
            self.eval()
            total_val_loss = 0
            correct_val_predictions = 0
            total_val_predictions = 0
            
            with torch.no_grad():
                for batch in val_loader:
                    outputs = self(batch)
                    target = batch['target'].view(-1, 1)  # Add this line
                    loss = criterion(outputs, target)
                    
                    total_val_loss += loss.item()
                    predictions = (outputs >= 0.5).float()
                    correct_val_predictions += (predictions == target).sum().item()  # Update this line
                    total_val_predictions += target.size(0)
            
            avg_val_loss = total_val_loss / len(val_loader)
            val_accuracy = correct_val_predictions / total_val_predictions
            
            scheduler.step(avg_val_loss)
            
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                best_model = self.state_dict().copy()
                patience_counter = 0
            else:
                patience_counter += 1
                
            print(f'Epoch {epoch+1}/{num_epochs}')
            print(f'Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
            print(f'Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')
            print(f'Patience Counter: {patience_counter}/{patience}')
            print('--------------------')
            
            if patience_counter >= patience:
                print(f'Early stopping triggered after epoch {epoch+1}')
                break
        
        self.load_state_dict(best_model)
        return self, {
            'best_val_loss': best_val_loss,
            'epochs_trained': epoch + 1
        }

def create_feature_filtered_loader(original_loader, feature_names, batch_size):
    filtered_games = []

    for game in original_loader.dataset.game_objects:
        # Check for NaN values in all sequences
        has_nan = (game.matchups[feature_names].isna().any().any() or 
                  game.team_history[feature_names].isna().any().any() or 
                  game.opponent_history[feature_names].isna().any().any())
        
        if not has_nan:  # Only include games without NaN values
            filtered_game = GameData(
                game_id=game.game_id,
                game_date=game.game_date,
                is_regular_season=game.is_regular_season,
                is_playoffs=game.is_playoffs,
                is_pre_season=game.is_pre_season,
                matchups=game.matchups[feature_names],
                team_history=game.team_history[feature_names],
                opponent_history=game.opponent_history[feature_names],
                target=game.target
            )
            filtered_games.append(filtered_game)
    
    print(f"Filtered out {len(original_loader.dataset.game_objects) - len(filtered_games)} games with NaN values")
    print(f"Remaining games: {len(filtered_games)}")

    filtered_dataset = NBAGamesDataset(filtered_games)

    return DataLoader(
        filtered_dataset,
        batch_size=batch_size,
        shuffle=True if original_loader.dataset == 'train' else False,
        collate_fn=GameData.collate_batch
    )

Train the model and check accuracies

In [10]:
import pickle

with open('processed_game_objects.pkl', 'rb') as f:
    processed_data = pickle.load(f)

features = [
    'days_ago', 
    'games_ago', 
    'is_home_team', 
    'is_regular_season_matchup', 
    'is_playoffs_matchup', 
    'is_pre_season_matchup', 
    'wl', 
    'pts_for', 
    'fg_pct_for', 
    'fg3_pct_for', 
    'fg3m_for', 
    'ft_pct_for', 
    'ftm_for', 
    'reb_for', 
    'ast_for', 
    'stl_for', 
    'blk_for', 
    'tov_for', 
    'pts_against', 
    'fg_pct_against', 
    'fg3_pct_against', 
    'fg3m_against', 
    'ft_pct_against', 
    'ftm_against', 
    'reb_against', 
    'ast_against', 
    'stl_against', 
    'blk_against', 
    'tov_against'
    ]

train_loader = create_feature_filtered_loader(train_loader, features, batch_size=32)
val_loader = create_feature_filtered_loader(val_loader, features, batch_size=32)

# Create and train model
model = PredictorRNN(feature_names=features)
trained_model, history = model.train_model(train_loader, val_loader)

Filtered out 10 games with NaN values
Remaining games: 23032
Filtered out 0 games with NaN values
Remaining games: 2880
Epoch 1/50
Train Loss: 0.6619, Train Accuracy: 0.6104
Val Loss: 0.6780, Val Accuracy: 0.5740
Patience Counter: 0/5
--------------------
Epoch 2/50
Train Loss: 0.6553, Train Accuracy: 0.6151
Val Loss: 0.6799, Val Accuracy: 0.5694
Patience Counter: 1/5
--------------------
Epoch 3/50
Train Loss: 0.6545, Train Accuracy: 0.6187
Val Loss: 0.6819, Val Accuracy: 0.5608
Patience Counter: 2/5
--------------------
Epoch 4/50
Train Loss: 0.6542, Train Accuracy: 0.6169
Val Loss: 0.6807, Val Accuracy: 0.5601
Patience Counter: 3/5
--------------------
Epoch 5/50
Train Loss: 0.6534, Train Accuracy: 0.6195
Val Loss: 0.6806, Val Accuracy: 0.5576
Patience Counter: 4/5
--------------------
Epoch 6/50
Train Loss: 0.6529, Train Accuracy: 0.6181
Val Loss: 0.6833, Val Accuracy: 0.5583
Patience Counter: 5/5
--------------------
Early stopping triggered after epoch 6


Evaluate the model

In [11]:
def evaluate_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    correct_predictions = 0
    total_predictions = 0
    all_predictions = []
    all_targets = []
    
    with torch.no_grad():  # No need to track gradients
        for batch in test_loader:
            # Forward pass
            outputs = model(batch)
            target = batch['target'].view(-1, 1)
            
            # Get predictions
            predictions = (outputs >= 0.5).float()
            
            # Track accuracy
            correct_predictions += (predictions == target).sum().item()
            total_predictions += target.size(0)
            
            # Store predictions and targets for additional metrics
            all_predictions.extend(predictions.cpu().numpy().flatten())
            all_targets.extend(target.cpu().numpy().flatten())
    
    # Calculate accuracy
    accuracy = correct_predictions / total_predictions
    
    print(f"\nTest Results:")
    print(f"Total Test Games: {total_predictions}")
    print(f"Correct Predictions: {correct_predictions}")
    print(f"Test Accuracy: {accuracy:.4f}")
    
    return accuracy, all_predictions, all_targets

# Usage:
test_loader = create_feature_filtered_loader(test_loader, features, batch_size=32)
accuracy, predictions, targets = evaluate_model(trained_model, test_loader)

Filtered out 0 games with NaN values
Remaining games: 2881

Test Results:
Total Test Games: 2881
Correct Predictions: 1657
Test Accuracy: 0.5751
